# Задание по теме «Коллаборативная фильтрация»

ПАКЕТ SURPRISE

    используйте данные MovieLens 1M
    можно использовать любые модели из пакета
    получите RMSE на тестовом сете 0.87 и ниже


In [1]:
# Устанавливаем пакет
!pip3 install surprise

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
# Подключаем библиотеки
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [3]:
# Загружаем данные
mnames = ['movie_id', 'title', 'genres']
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames)
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames)


/Users/vandal/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py:765: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


Создаем датасет для обучения

In [4]:
movies_with_ratings = movies.join(ratings.set_index('movie_id'), on='movie_id').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
# Датасет для обучения
dataset = pd.DataFrame({
    'uid': movies_with_ratings.user_id,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [6]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [7]:
ratings.rating.min()

1

In [8]:
ratings.rating.max()

5

In [9]:
# Готовим данные для обучения
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [11]:
# Создаем выборки
trainset, testset = train_test_split(data, test_size=.15)

In [16]:
# Обучаем модель
model = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
model.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [17]:
test_pred = model.test(testset)

In [18]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8563


0.8563113712780546

RMSE удовлетворяет условиям задачи

Попробуем улучшить

In [20]:
# Попробуем модель с z нормализацией

from surprise import KNNWithZScore
model_new = KNNWithZScore(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
model_new.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [21]:
test_pred_new = model_new.test(testset)

In [22]:
accuracy.rmse(test_pred_new, verbose=True)

RMSE: 0.8568


0.8567895822060035

Качество не изменилось

In [23]:
# Попробуем взять больше соседей

model_more_nn = KNNWithMeans(k=70, sim_options={'name': 'pearson_baseline', 'user_based': False})
model_more_nn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [24]:
test_pred_more_nn = model_more_nn.test(testset)

In [25]:
accuracy.rmse(test_pred_more_nn, verbose=True)

RMSE: 0.8585


0.858546211447527

Качество стало немного ниже

### Выводы:
Качество не улучшается, а только снижается. Но и RMSE в 0.8563 - весьма большая ошибка. 